In [6]:
import yaml
import os
import pathlib
import time
from Agent.xwAPI import xwBackend
from utils.construct_prompt import get_api_doc
from utils.ChatGPT import ChatGPT

In [7]:
import evaluate

In [8]:
CONFIG_FILE = "./config/config.yaml"

In [4]:
def set_agent_configuration(configuration_file_path):
    with open(configuration_file_path, mode='r') as file:
        config = yaml.load(file, Loader=yaml.Loader)

    agent_config = config["Agent"]
    agent_config['ChatGPT_1']['api_keys'] = [os.environ['OPENAI_API_KEY']]
    return agent_config

In [5]:
def get_api_documentation(agent_config):
    with open(agent_config['api_doc_path']) as f:
        api_doc = yaml.load(f, Loader=yaml.FullLoader)
    return api_doc

In [6]:
def get_excel_backend(agent_config, api_doc):
    if agent_config['API_backend'] == 'xw':
        xw_backend = xwBackend(agent_config['APP_backend'], api_doc)
    return xw_backend

In [7]:
def create_path_generator(file_path):
    return pathlib.Path(f"{file_path}").glob("**/*")

In [8]:
def get_source_excel_filenames(path_generator):
    response_log_paths = [str(path).split("\\")[-1] for path in path_generator if path.is_file()]
    return [f"{log_file[2:-5]}.xlsx" for log_file in response_log_paths]

In [9]:
agent_config = set_agent_configuration(configuration_file_path=CONFIG_FILE)
api_doc = get_api_documentation(agent_config=agent_config)
xw_backend = get_excel_backend(agent_config=agent_config, api_doc=api_doc)

In [ ]:
path_generator = create_path_generator(file_path="../output_dir/refined_responses/")
input_file_paths = [str(path) for path in path_generator]
input_file_paths

In [ ]:
path_generator = create_path_generator(file_path="../output_dir/refined_responses/")
excel_file_names = get_source_excel_filenames(path_generator=path_generator)
excel_file_paths = [f"../dataset/task_sheets/{file_name}" for file_name in excel_file_names]
excel_file_paths

In [ ]:
example_index = 1
example_src_file_path = excel_file_paths[example_index]
example_input_file_path = input_file_paths[example_index]
example_src_file_path, example_input_file_path

In [ ]:
path_generator = create_path_generator(file_path="../output_dir/intermediate_responses/")
correct_response_file_paths = [str(path) for path in path_generator]
correct_response_file_paths

In [ ]:
example_correct_file_path = correct_response_file_paths[example_index]
example_correct_file_path

In [ ]:
test_index = 2
test_input_file_path = input_file_paths[test_index]
test_src_file_path = excel_file_paths[test_index]
test_input_file_path, test_src_file_path

In [16]:
def get_sheet_state(file_path):
    if file_path is not None:
        time.sleep(0.5)
        xw_backend.OpenWorkbook(file_path)

    return xw_backend.GetSheetsState()

In [ ]:
example_sheet_state = get_sheet_state(file_path=example_src_file_path)
example_sheet_state

In [18]:
def extract_docs_for_input_functions(input_file_path, agent_config):
    with open(input_file_path) as file:
        input_functions = yaml.load(file, Loader=yaml.Loader)
    prompt_format = agent_config['ChatGPT_1']['prompt_format']
    api_list, api_usage, api_detail_doc = get_api_doc(prompt_format, api_doc)
    # Filter upto first open paranthesis
    unique_function_names = set([function[:function.find("(")] for function in input_functions["refined_response"]])
    documentation_for_functions = [api_detail_doc[name] for name in unique_function_names]
    return documentation_for_functions

In [ ]:
extract_docs_for_input_functions(input_file_path=example_input_file_path, agent_config=agent_config)

In [20]:
def get_input_functions(input_file_path):
    with open(input_file_path) as file:
        input_functions = yaml.load(file, Loader=yaml.Loader)
    return yaml.dump(input_functions["refined_response"])

In [21]:
def get_correct_summarization(correct_file_path):
    with open(correct_file_path) as file:
        correct_summarizations = yaml.load(file, Loader=yaml.Loader)
    return yaml.dump(correct_summarizations["intermediate response"])

In [ ]:
prompt = """
SYSTEM
Summarize the each sub-step of instructions into explanations in natural language.

USER
{input_example}
Here is the supplementary documentation you can reference:
{documentation_example}
Here is the corresponding sheet state:
{sheet_state_example}

ASSISTANT
{correct_example}

USER
{actual_input}
Here is the supplementary documentation you can reference:
{actual_documentation}
Here is the corresponding sheet state:
{actual_sheet_state}
"""
prompt

In [ ]:
prompt = prompt.format(input_example=get_input_functions(example_input_file_path),
                       documentation_example=extract_docs_for_input_functions(input_file_path=example_input_file_path, agent_config=agent_config),
                       correct_example=get_correct_summarization(correct_file_path=example_correct_file_path),
                       sheet_state_example=get_sheet_state(file_path=example_src_file_path),
                       actual_input=get_input_functions(test_input_file_path),
                       actual_documentation=extract_docs_for_input_functions(input_file_path=test_input_file_path, agent_config=agent_config),
                       actual_sheet_state=get_sheet_state(file_path=test_src_file_path)
                       )
print(prompt)

In [24]:
async def call_chat_gpt():
    try:
        chatbot = ChatGPT(agent_config['ChatGPT_1'], context=[], interaction_mode=True)
        response = await chatbot(prompt)
    except Exception as e:
        print(f"error occurs when parsing response: {e}")
    else:
        return response


In [ ]:
response = await call_chat_gpt()

In [ ]:
print(response)